In [ ]:
!pip install peewee

     |████████████████████████████████| 855 kB 4.2 MB/s 
  Created wheel for peewee: filename=peewee-3.14.10-cp37-cp37m-linux_x86_64.whl size=616214 sha256=7a462e293b21b5af61b3b5aed63ad0c0f9fd54b24ded85e5514f2385b9ccdfc6
  Stored in directory: /root/.cache/pip/wheels/df/4f/73/3228affbec8e2e73e622306ac47ebba39df2170bd784eec69c
Successfully built peewee


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import peewee

db_features_one = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_one.db")
db_features_two = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_two.db")
db_features_three = peewee.SqliteDatabase("/content/drive/MyDrive/diss_data/features_three.db")


class ExtractedFeaturesOne(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_one
        legacy_table_names = False


class ExtractedFeaturesTwo(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_two
        legacy_table_names = False


class ExtractedFeaturesThree(peewee.Model):
    user_id = peewee.IntegerField()
    data_split_type = peewee.IntegerField()
    feature_array = peewee.BlobField()

    class Meta:
        database = db_features_three
        legacy_table_names = False


db_features_one.create_tables(
    [
        ExtractedFeaturesOne,
        ExtractedFeaturesTwo,
        ExtractedFeaturesThree
    ]
)


/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
from tqdm import tqdm, trange
from imblearn.over_sampling import SMOTE
import json
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline 
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
import pickle


In [ ]:
f = open("/content/drive/MyDrive/diss_data/suspicious_network_two.json")
sus_raw = json.load(f)
sus = [int(sus) for sus in sus_raw]
f.close()


In [ ]:
file = open("/content/drive/MyDrive/diss_data/pipe_network_two.obj",'rb')
pipe = pickle.load(file)
file.close()


In [ ]:
def get_data(query, X, y):
  for feature in tqdm(query):
    X.append(json.loads(feature.feature_array.decode('utf-8')))
    y.append(feature.user_id)


In [ ]:
X = []
y = []


In [ ]:
query = ExtractedFeaturesOne.select().where(~(ExtractedFeaturesOne.user_id << sus))
get_data(query, X, y)


100%|██████████| 623064/623064 [00:24<00:00, 25471.02it/s]


In [ ]:
query = ExtractedFeaturesTwo.select().where(~(ExtractedFeaturesTwo.user_id << sus))
get_data(query, X, y)


100%|██████████| 621922/621922 [00:24<00:00, 25226.35it/s]


In [ ]:
query = ExtractedFeaturesThree.select().where(~(ExtractedFeaturesThree.user_id << sus))
get_data(query, X, y)


100%|██████████| 623904/623904 [00:26<00:00, 23951.56it/s]


In [ ]:
if any(s in y for s in sus):
    raise Exception("SUS!")


In [ ]:
len(X)


1868890

In [ ]:
predictions_dict = {item:[] for item in set(y)}

for pair in tqdm(zip(X, y)):
    _x, _y = pair
    predictions_dict[_y].append(pipe.predict([_x])[0])


1868890it [08:49, 3529.04it/s]


In [ ]:
from collections import Counter
counts = {key:Counter(value) for key, value in predictions_dict.items()}

In [ ]:
confidences = {item:[] for item in set(y)}

for user, prediction_count in counts.items():
    sorted_ = sorted(prediction_count.items(), key=lambda item: item[1])
    total = sum([item[1] for item in sorted_])
    confidence = sorted_[0][1] / total
    confidences[user] = {"u": sorted_[0][0], "c": confidence}


In [ ]:
confidences

{1: {'c': 0.013289036544850499, 'u': 49687},
 2: {'c': 0.008955223880597015, 'u': 12062},
 3: {'c': 0.005494505494505495, 'u': 27814},
 4: {'c': 0.0035714285714285713, 'u': 13348},
 5: {'c': 0.00796812749003984, 'u': 27814},
 6: {'c': 0.009911894273127754, 'u': 2395},
 7: {'c': 0.005917159763313609, 'u': 22586},
 8: {'c': 0.010526315789473684, 'u': 15164},
 9: {'c': 0.004545454545454545, 'u': 15654},
 10: {'c': 0.003857280617164899, 'u': 12062},
 11: {'c': 0.006389776357827476, 'u': 2395},
 12: {'c': 0.003968253968253968, 'u': 22586},
 13: {'c': 0.006433823529411764, 'u': 49687},
 14: {'c': 0.009652509652509652, 'u': 15654},
 15: {'c': 0.00625, 'u': 2395},
 16: {'c': 0.006134969325153374, 'u': 27814},
 17: {'c': 0.006134969325153374, 'u': 22502},
 18: {'c': 0.002840909090909091, 'u': 49687},
 19: {'c': 0.004219409282700422, 'u': 49687},
 20: {'c': 0.010736196319018405, 'u': 29315},
 21: {'c': 0.00651890482398957, 'u': 49735},
 22: {'c': 0.011138613861386138, 'u': 12062},
 23: {'c': 0.0

In [ ]:
for user, pair in confidences.items():
    if pair["c"] >= 0.02:
        print(user, ":" , pair)


649 : {'u': 15164, 'c': 0.02}
701 : {'u': 24937, 'c': 0.02}
66770 : {'u': 22586, 'c': 0.02}
1635 : {'u': 15095, 'c': 0.02}
34864 : {'u': 6439, 'c': 0.02}
2143 : {'u': 19272, 'c': 0.02}
2153 : {'u': 15164, 'c': 0.02}
100614 : {'u': 15095, 'c': 0.02}
2451 : {'u': 24937, 'c': 0.02}
69865 : {'u': 12062, 'c': 0.02}
4482 : {'u': 15095, 'c': 0.02}
6240 : {'u': 27814, 'c': 0.02}
72610 : {'u': 14311, 'c': 0.02}
105493 : {'u': 24937, 'c': 0.02}
43484 : {'u': 27814, 'c': 0.02}
43844 : {'u': 15654, 'c': 0.02}
44520 : {'u': 24937, 'c': 0.02}
45103 : {'u': 15095, 'c': 0.02}
45503 : {'u': 27814, 'c': 0.02}
45849 : {'u': 22502, 'c': 0.02}
13246 : {'u': 22502, 'c': 0.02}
46145 : {'u': 15918, 'c': 0.02}
46272 : {'u': 15654, 'c': 0.020967741935483872}
46385 : {'u': 22586, 'c': 0.02}
46977 : {'u': 6439, 'c': 0.02}
14617 : {'u': 6439, 'c': 0.020618556701030927}
14833 : {'u': 2395, 'c': 0.023809523809523808}
47881 : {'u': 29315, 'c': 0.02}
80762 : {'u': 24937, 'c': 0.02}
15357 : {'u': 27814, 'c': 0.02}
1659